# Topic 3. Algebraic data types

## 3.2 Data types

Auxiliary definitions:

In [ ]:
trait Isomorphic[A, B]{
    
    def from(a: A): B
    
    def to(b: B): A
    
    // equality 
    
    def equalA(a1: A, a2: A): Boolean = 
        a1 == a2
    
    def equalB(b1: B, b2: B): Boolean =
        b1 == b2
    
    // Bijection laws
    
    def law1(a: A): Boolean = 
        equalA(to(from(a)), a)
    
    def law2(b: B): Boolean = 
        equalB(from(to(b)), b)
}

## Exercise 1

### Part a)

Prove that the isomorphism $Either[Unit, Unit] \cong Boolean$ holds by implementing the following bijections: 

In [ ]:
object Iso extends Isomorphic[Boolean, Either[Unit, Unit]]{

    def from(a: Boolean): Either[Unit, Unit] = 
        if (a) Right(()) 
        else Left(())    
    
    def to(a: Either[Unit, Unit]): Boolean = 
        a match {
            case Left(_) => false
            case Right(_) => true
        }
}

Check that they are indeed mutual inverses, i.e. that for all $a: Boolean$, `toBoolean(fromBoolean(a))==a`, and that for all $a: Either[Unit, Unit]$, `fromBoolean(toBoolean(a))==a`.

In [ ]:
Iso.law1(true)
Iso.law1(false)
Iso.law2(Left(()))
Iso.law2(Right(()))


### Part b)

Show that we can redefine `Option[A]` using `Either[A,Unit]`: 

In [ ]:
class Iso[A] extends Isomorphic[Option[A], Either[A, Unit]]{
    
    def from(o: Option[A]): Either[A, Unit] = 
        o match {
            case None => Right(())
            case Some(a) => Left(a)
        }

    def to(e: Either[A, Unit]): Option[A] = 
        e match {
            case Left(a) => Some(a)
            case Right(()) => None
        }
}

Check that these functions are mutual inverses. For that, fix $A$ to particular types (e.g. `Boolean`, `Int`, etc.), and test the equivalences `from(to(e)) == e` and `to(from(o)) == o` for some values $o$ and $e$.

In [ ]:
(new Iso[Int]).law1(None)
(new Iso[Boolean]).law1(None)
(new Iso[Int]).law1(Some(1))
(new Iso[Boolean]).law1(Some(true))
(new Iso[Int]).law2(Left(1))
(new Iso[Int]).law2(Right(()))

## Exercise 2

How many functions are there of type `Either[Unit, Either[Unit, Unit]] => Boolean`? Identify all of them as alternative implementations of the following signature: 

In [ ]:
def f1(e: Either[Unit, Either[Unit, Unit]]): Boolean = 
    e match {
        case Left(()) => ??? // true or false
        case Right(Left(())) => ??? // true or false
        case Right(Right(())) => ??? // true or false
    }

In [ ]:
def f1(e: Either[Unit, Either[Unit, Unit]]): Boolean = 
    e match {
        case Left(()) => false // true or false
        case Right(Left(())) => false // true or false
        case Right(Right(())) => false // true or false
    }

In [ ]:
def f1(e: Either[Unit, Either[Unit, Unit]]): Boolean = 
    e match {
        case Left(()) => true // true or false
        case Right(Left(())) => false // true or false
        case Right(Right(())) => false // true or false
    }

etc. (eight functions in total)

Idem, as alternative lambda expressions:

In [ ]:
val f1: Either[Unit, Either[Unit, Unit]] => Boolean = 
    {
        case Left(()) => ??? // true or false
        case Right(Left(())) => ???// true or false
        case Right(Right(())) => ???// true or false
    }

## Exercise 3

How many different implementations are there of the following function-method? Recall that two implementations will be considered different if the corresponding mathematical functions are different.

In [ ]:
def f1(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    Left(())

def f2(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    Right(Left(()))

def f3(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    Right(Right(()))

def f4(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    if (b) Left(()) else Right(Left())

def f5(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    if (b) Right(Left()) else Right(Right(()))

def f6(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    if (b) Left(()) else Right(Right(()))

def f7(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    if (!b) Left(()) else Right(Left())

def f8(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    if (!b) Right(Left()) else Right(Right(()))

def f9(b: Boolean): Either[Unit, Either[Unit, Unit]] = 
    if (!b) Left(()) else Right(Right(()))


## Exercise 4

Show that the following law holds for exponent types: $X => (Y => Z) \cong (Y, X) => Z$, for all types $X$, $Y$ and $Z$.

In [ ]:
class Iso[X, Y, Z] extends Isomorphic[X => (Y => Z), (Y, X) => Z]{
    
    // uncurry
    def from(f: X => Y => Z): (Y, X) => Z = 
        (y, x) => f(x)(y)
 
    // curry
    def to(f: (Y, X) => Z): X => Y => Z = 
        x => y => f(y, x)
}

Implement function equality for the following signatures and check that both functions, `curry` and `uncurry`, are inverses of each other for two sample functions $ex1$ and $ex2$:  

In [ ]:
def ex1: Boolean => Boolean => Boolean = b1 => b2 => false

def ex2: (Boolean, Boolean) => Boolean = (b1, b2) => true

Accordingly, we need to override the equality functions:

In [ ]:
object Iso extends Iso[Boolean, Boolean, Boolean]{
    
    override def equalA(f1: Boolean => Boolean => Boolean, 
               f2: Boolean => Boolean => Boolean): Boolean = 
        f1(false)(false) == f2(false)(false) &&
        f1(false)(true) == f2(false)(true) &&
        f1(true)(false) == f2(true)(false) &&
        f1(true)(true) == f2(true)(true)
    
    override def equalB(f1: (Boolean, Boolean) => Boolean, 
               f2: (Boolean, Boolean) => Boolean): Boolean = 
        f1(false,false) == f2(false,false) &&
        f1(false,true) == f2(false,true) &&
        f1(true,false) == f2(true,false) &&
        f1(true,true) == f2(true,true)
}

Now, check that curry and uncurry are inverses of each other for sample
functions `ex1` and `ex2`:


In [ ]:
Iso.law1(ex1)
Iso.law2(ex2)

## Exercise 5

Shows that the following law holds for exponent types: $X => (Y, Z) \cong (X => Y, X => Z)$, for all types $X$, $Y$ and $Z$.

In [ ]:
class Iso[X, Y, Z] extends Isomorphic[X => (Y, Z), (X => Y, X => Z)]{
    
    def from(f: X => (Y, Z)): (X => Y, X => Z) = 
        (x => f(x)._1, x => f(x)._2)

    def to(t: (X => Y, X => Z)): X => (Y, Z) = t match {
        case (f1, f2) => x => (f1(x), f2(x))
    }
}

Fix $X$, $Y$ and $Z$ to particular types, implement equality for the corresponding signatures and check that both functions, `from` and `to`, are inverses of each other given two sample functions of your choice.  

In [ ]:
val ex1: Boolean => (Boolean, Boolean) = 
    _ => (false, false)

val ex2: (Boolean => Boolean, Boolean => Boolean) = 
    (_ => true, ! _)

In [ ]:
object Iso extends Iso[Boolean, Boolean, Boolean]{
    
    def equalAux(f1: Boolean => Boolean, f2: Boolean => Boolean): Boolean = 
        f1(false) == f2(false) && 
        f1(true) == f2(true)
    
    override def equalA(f1: Boolean => (Boolean, Boolean), f2: Boolean => (Boolean, Boolean)): Boolean = 
        f1(false) == f2(false) && 
        f1(true) == f2(true)
    
    override def equalB(f1: (Boolean => Boolean, Boolean => Boolean), 
                        f2: (Boolean => Boolean, Boolean => Boolean)): Boolean = 
        equalAux(f1._1, f2._1) && equalAux(f1._2, f2._2)
}

In [ ]:
Iso.law1(ex1)
Iso.law2(ex2)

## Exercise 6

Shows that the following isomorphisms holds for algebraic data types:

$$ 
\begin{array}{rcl}
Either[X,Nothing] & \cong & X \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
Either[X,Y] & \cong & Either[Y,X] \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
Either[X,Either[Y,Z]] & \cong & Either[Either[X,Y],Z] \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
(X,Nothing) & \cong & Nothing \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
(X,Unit) & \cong & X \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
(X,Y) & \cong & (Y,X) \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
(X,(Y,Z)) & \cong & ((X,Y),Z) \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
(X,Either[Y,Z]) & \cong & Either[(X,Y),(X,Z)] \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

$$ 
\begin{array}{rcl}
Nothing  \Rightarrow X & \cong & Unit \\
\end{array}
$$



In [ ]:
// extends trait Isomorphic

# Ejercicio 7

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(1+1)^X \cong Boolean^X$

A continuación se muestran unos casos de prueba de este isomorfismo para $X=Int$:

In [ ]:
class IsoTest(
    from: (Int => Either[Unit, Unit]) => Int => Boolean, 
    to: (Int => Boolean) => Int => Either[Unit, Unit]
) extends FlatSpec with Matchers{
    
    val f: Int => Either[Unit, Unit] = 
        i => if (i % 2 == 0) Left(()) else Right(())
    
    val g: Int => Boolean = 
        _ % 2 == 0
    
    "from-to" should "work" in {
        from(to(g))(0) shouldBe g(0)
        from(to(g))(1) shouldBe g(1)
        from(to(g))(2) shouldBe g(2)
        from(to(g))(3) shouldBe g(3)
    }
    
    "to-from" should "work" in {
        to(from(f))(0) shouldBe f(0)
        to(from(f))(1) shouldBe f(1)
        to(from(f))(2) shouldBe f(2)
        to(from(f))(3) shouldBe f(3)
    }
}

In [ ]:
// extends trait Isomorphic

# Ejercicio 8

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$, $Y$ y $Z$: 

$(1+Y+Z+Y*Z)^X \cong (Y+1)^X * (Z+1)^X$

A continuación se muestran unos casos de prueba de este isomorfismo para $X=Int$, $Y=Unit$ y $Z=Unit$:

In [ ]:
class IsoTest(
    from: (Int => Either[Unit, Either[Unit, Either[Unit, (Unit, Unit)]]]) => (Int => Option[Unit], Int => Option[Unit]), 
    to: ((Int => Option[Unit], Int => Option[Unit])) => Int => Either[Unit, Either[Unit, Either[Unit, (Unit, Unit)]]]
) extends FlatSpec with Matchers{
    
    val f: Int => Either[Unit, Either[Unit, Either[Unit, (Unit, Unit)]]] = 
        i => if (i % 4 == 0) Left(()) 
             else if (i % 4 == 1) Right(Left(()))
             else if (i % 4 == 2) Right(Right(Left(())))
             else Right(Right(Right(((),()))))
    
    val g: (Int => Option[Unit], Int => Option[Unit]) = 
        (i => if (i % 2 == 0) None else Some(()),
         i => if (i % 2 == 0) None else Some(()))
    
    "from-to" should "work" in {
        from(to(g))._1(0) shouldBe g._1(0)
        from(to(g))._1(1) shouldBe g._1(1)
        from(to(g))._1(2) shouldBe g._1(2)
        from(to(g))._1(3) shouldBe g._1(3)
        from(to(g))._2(0) shouldBe g._2(0)
        from(to(g))._2(1) shouldBe g._2(1)
        from(to(g))._2(2) shouldBe g._2(2)
        from(to(g))._2(3) shouldBe g._2(3)
    }
    
    "to-from" should "work" in {
        to(from(f))(0) shouldBe f(0)
        to(from(f))(1) shouldBe f(1)
        to(from(f))(2) shouldBe f(2)
        to(from(f))(3) shouldBe f(3)
    }
}

In [ ]:
// extends trait Isomorphic